In [1]:
%cd '/Users/abhilashdhal/Projects/'

/Users/abhilashdhal/Projects


In [2]:
import pandas as pd
from importlib import reload
import src.Engines.analysis_engine as analysis_engine
import src.Connectors.gcp_bigquery_utils as gcp_bigquery_utils
reload(analysis_engine)
reload(gcp_bigquery_utils)

<module 'src.Connectors.gcp_bigquery_utils' from '/Users/abhilashdhal/Projects/src/Connectors/gcp_bigquery_utils.py'>

## 1. Download Dataset from BigQuery for a given Primary Diagnosis By Primary Site and the Normal Tissue for the Primary site

In [3]:
project_id = 'rnaseqml'
dataset_id = 'rnaseqexpression'
table_id = 'expr_clustered_08082024'
bq_queries = gcp_bigquery_utils.BigQueryQueries(project_id=project_id, 
                                              dataset_id=dataset_id,
                                              table_id=table_id)



I0000 00:00:1725303525.127815  712075 config.cc:230] gRPC experiments enabled: call_status_override_on_cancellation, event_engine_dns, event_engine_listener, http2_stats_fix, monitoring_experiment, pick_first_new, trace_record_callops, work_serializer_clears_time_cache
I0000 00:00:1725303525.133415  712075 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


In [4]:
pr_site = 'Head and Neck'
pr_diag = 'Squamous cell carcinoma, NOS'
data_from_bq = bq_queries.get_df_for_pydeseq(primary_site=pr_site, primary_diagnosis=pr_diag)

I0000 00:00:1725303526.602047  712075 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


In [5]:
data_from_bq.head()

,case_id,primary_site,sample_type,tissue_type,primary_diagnosis,expr_unstr_count
0,5422a86d-f9ff-4fb6-873e-bf7cc3de50e5,Head and Neck,Primary Tumor,Tumor,"Squamous cell carcinoma, NOS","[1666, 0, 1310, 663, 367, 598, 10862, 1930, 20..."
1,2855bf40-e5e1-4519-b47a-327bfc275279,Head and Neck,Primary Tumor,Tumor,"Squamous cell carcinoma, NOS","[1597, 1, 1467, 544, 365, 345, 1172, 1256, 620..."
2,0fb3e5ff-54f0-43c5-9322-541bb7825e7f,Head and Neck,Primary Tumor,Tumor,"Squamous cell carcinoma, NOS","[977, 35, 2057, 539, 247, 1997, 6023, 2071, 12..."
3,3c5bc678-41e9-46b3-9686-5cd95659ab71,Head and Neck,Primary Tumor,Tumor,"Squamous cell carcinoma, NOS","[5433, 0, 2240, 2600, 3159, 402, 4090, 849, 22..."
4,50239d1d-b3a4-4240-ab83-b24ba67b1299,Head and Neck,Primary Tumor,Tumor,"Squamous cell carcinoma, NOS","[1211, 0, 2168, 325, 320, 114, 2900, 2571, 153..."


## 2. Data Preprocessing for PyDeSeq and GSEA

In [18]:
analysis_eng = analysis_engine.AnalysisEngine(data_from_bq, analysis_type='DE')
if not analysis_eng.check_tumor_normal_counts():
    raise ValueError("Tumor and Normal counts should be at least 10 each")
gene_ids_or_gene_cols_df = pd.read_csv('/Users/abhilashdhal/Projects/personal_docs/data/Transcriptomics/data/gene_annotation/gene_id_to_gene_name_mapping.csv')
gene_ids_or_gene_cols = list(gene_ids_or_gene_cols_df['gene_id'].to_numpy())

In [19]:
exp_df = analysis_eng.expand_data_from_bq(data_from_bq, gene_ids_or_gene_cols=gene_ids_or_gene_cols, analysis_type='DE')
metadata = analysis_eng.metadata_for_pydeseq(exp_df=exp_df)
counts_for_de = analysis_eng.counts_from_bq_df(exp_df, gene_ids_or_gene_cols)

## 3. Run DESeq2 for the given Primary Diagnosis By Primary Site and the Normal Tissue for the Primary site

In [20]:
res_pydeseq = analysis_eng.run_pydeseq(metadata=metadata, counts=counts_for_de)


03/09//2024 12:30:1725303653 AM - INFO - PyDeSeqWrapper.run_deseq: Running DESeq2 for groups: {'group1': 'Tumor', 'group2': 'Normal'}
03/09//2024 12:30:1725303653 AM - INFO - PyDeSeqWrapper.run_deseq: Running DESeq2  factor analysis with design factor: C and o
03/09//2024 12:30:1725303653 AM - INFO - PyDeSeqWrapper.run_deseq: Statistical analysis of Tumor vs Normal in {'group1': 'Tumor', 'group2': 'Normal'}


Fitting size factors...
... done in 0.42 seconds.

I0000 00:00:1725303657.111200  712075 work_stealing_thread_pool.cc:320] WorkStealingThreadPoolImpl::PrepareFork
I0000 00:00:1725303657.131892  712075 work_stealing_thread_pool.cc:320] WorkStealingThreadPoolImpl::PrepareFork
I0000 00:00:1725303657.135921  712075 work_stealing_thread_pool.cc:320] WorkStealingThreadPoolImpl::PrepareFork
I0000 00:00:1725303657.144212  712075 work_stealing_thread_pool.cc:320] WorkStealingThreadPoolImpl::PrepareFork
I0000 00:00:1725303657.150960  712075 work_stealing_thread_pool.cc:320] WorkStealingThreadPoolImpl::PrepareFork
I0000 00:00:1725303657.160985  712075 work_stealing_thread_pool.cc:320] WorkStealingThreadPoolImpl::PrepareFork
I0000 00:00:1725303657.170410  712075 work_stealing_thread_pool.cc:320] WorkStealingThreadPoolImpl::PrepareFork
I0000 00:00:1725303657.190287  712075 work_stealing_thread_pool.cc:320] WorkStealingThreadPoolImpl::PrepareFork
I0000 00:00:1725303657.267184  712075 work_stealing_t

In [ ]:
res_pydeseq_with_gene_names = pd.merge(res_pydeseq, gene_ids_or_gene_cols_df, left_on='index', right_on='gene_id')

## 4. Run GSEA for the given Primary Diagnosis By Primary Site and the Normal Tissue for the Primary site using a gene set database

In [ ]:
gene_set 
result, plot = analysis_eng.run_gsea(res_pydeseq, gene_set)

In [ ]:
# import src.ClassicML.DGE.pydeseq_utils as pydeseq_utils
# import pandas as pd
# from gseapy.plot import gseaplot
# import gseapy as gp
# import numpy as np
# import matplotlib.pyplot as plt
# from gseapy import dotplot
# ## Preprocess the data
# ## Load the count data saved from example 1.
# rna_seq_DGE_data  = pd.read_csv('./de_gsea_data/kidney_unstr_tumor_normal.csv')
# unique_data_by_case_id =  rna_seq_DGE_data.drop_duplicates(['case_id']).reset_index(drop=True)
# kidney_cancer_count_data = unique_data_by_case_id.iloc[:, :60660].T
# counts = kidney_cancer_count_data.copy().reset_index()
# counts = counts.set_index('index')
# counts = counts.T
# counts = pd.concat([unique_data_by_case_id[['case_id']], counts],axis=1)

# ## Run DE analysis
# pydeseq_obj = pydeseq_utils.PyDeSeqWrapper(count_matrix=counts, metadata=metadata, design_factors='Condition', groups = {'group1':'Tumor', 'group2':'Normal'})
# design_factor = 'Condition'
# result = pydeseq_obj.run_deseq(design_factor=design_factor, group1 = 'Tumor', group2 = 'Normal')

# ## Prepare the data for GSEA
# results_df = result.results_df
# results_df_filtered = results_df.dropna()
# results_df_filtered = results_df_filtered.reset_index()
# results_df_filtered['nlog10'] = -1*np.log10(results_df_filtered.padj)

# ## Create ranking for GSEA
# df = results_df_filtered.copy()
# df['Rank'] = -np.log10(df.padj)*df.log2FoldChange
# df = df.sort_values('Rank', ascending = False).reset_index(drop = True)
# ranking = df[['Gene', 'Rank']]
# pre_res = gp.prerank(rnk = ranking, gene_sets = 'RNA-Seq_Disease_Gene_and_Drug_Signatures_from_GEO', seed = 6, permutation_num = 100)

# ## Plot the GSEA results
# out = []
# for term in list(pre_res.results):
#     out.append([term,
#             pre_res.results[term]['fdr'],
#             pre_res.results[term]['es'],
#             pre_res.results[term]['nes']])

# out_df = pd.DataFrame(out, columns = ['Term','fdr', 'es', 'nes']).sort_values('fdr').reset_index(drop = True)
# terms = pre_res.res2d.Term
# axs = pre_res.plot(terms=terms[1])

# # Create dotplot of most enrichment terms from Gene Set
# ax = dotplot(pre_res.res2d,
#             column="FDR q-val",
#             title='KEGG_2016',
#             cmap=plt.cm.viridis,
#             size=6, # adjust dot size
#             figsize=(4,5), cutoff=0.25, show_ring=False)